# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), April 27, 2019**

In [1]:
using DataFrames

## Possible pitfalls

### Know what is copied when creating a `DataFrame`

In [2]:
x = DataFrame(rand(3, 5))

,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,0.845693,0.431133,0.40237,0.378532,0.505309
2,0.335637,0.363309,0.808961,0.253146,0.642472
3,0.836706,0.982497,0.541213,0.00189648,0.989832


In [3]:
y = convert(DataFrame, x)
x === y # no copyinng performed

true

In [4]:
y = copy(x)
x === y # not the same object

false

In [5]:
y = DataFrame(x)
x === y

false

In [6]:
all(x[i] === y[i] for i in ncol(x)) # the columns are also not the same

false

In [7]:
x = 1:3; y = [1, 2, 3]; df = DataFrame(x=x,y=y) # the same when creating arrays or assigning columns

,x,y
,Int64,Int64
1,1,1
2,2,2
3,3,3


In [8]:
y === df[:y] # different object

false

In [9]:
typeof(x), typeof(df[:x]) # range is converted to a vector

(UnitRange{Int64}, Array{Int64,1})

In [10]:
y === df[:, :y] # slicing rows always creates a copy

false

you can avoid copying by using `copycols=false` keyword argument in functions. In particular `DataFrame!` is a shorthand for a non-copying constructor.

In [11]:
df = DataFrame!(x=x,y=y)

,x,y
,Int64,Int64
1,1,1
2,2,2
3,3,3


In [12]:
y === df.y # now it is the same

true

In [13]:
select(df, :y)[1] === y # not the same

false

In [14]:
select(df, :y, copycols=false)[1] === y # the same

true

### Do not modify the parent of `GroupedDataFrame` or `view`

In [15]:
x = DataFrame(id=repeat([1,2], outer=3), x=1:6)
g = groupby(x, :id)

,id,x
,Int64,Int64
1,1,1
2,1,3
3,1,5
,id,x
,Int64,Int64
1,2,2
2,2,4
3,2,6


In [16]:
x[1:3, 1]=[2,2,2]
g # well - it is wrong now, g is only a view

,id,x
,Int64,Int64
1,2,1
2,2,3
3,1,5
,id,x
,Int64,Int64
1,2,2
2,2,4
3,2,6


In [17]:
s = view(x, 5:6, :)

,id,x
,Int64,Int64
1,1,5
2,2,6


In [18]:
deleterows!(x, 3:6)

,id,x
,Int64,Int64
1,2,1
2,2,2


In [19]:
s # error

BoundsError: BoundsError: attempt to access 2-element Array{Int64,1} at index [5:6]

### Remember that you can filter columns of a `DataFrame` using booleans

In [20]:
using Random
Random.seed!(1)
x = DataFrame(rand(5, 5))

,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,0.236033,0.210968,0.555751,0.209472,0.0769509
2,0.346517,0.951916,0.437108,0.251379,0.640396
3,0.312707,0.999905,0.424718,0.0203749,0.873544
4,0.00790928,0.251662,0.773223,0.287702,0.278582
5,0.488613,0.986666,0.28119,0.859512,0.751313


In [21]:
x[x[:x1] .< 0.25] # well - we have filtered columns not rows by accident as you can select columns using booleans

,x1,x4
,Float64,Float64
1,0.236033,0.209472
2,0.346517,0.251379
3,0.312707,0.0203749
4,0.00790928,0.287702
5,0.488613,0.859512


In [22]:
x[x[:x1] .< 0.25, :] # probably this is what we wanted

,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,0.236033,0.210968,0.555751,0.209472,0.0769509
2,0.00790928,0.251662,0.773223,0.287702,0.278582


### Single column selection for `DataFrame` creates aliases unless explicitly copied

In [23]:
x = DataFrame(a=1:3)
x[:b] = x[1] # alias
x[:c] = x[:, 1] # copy
x[:d] = x[1][:] # copy
x[:e] = copy(x[1]) # explicit copy
display(x)
x[1,1] = 100
display(x)

,a,b,c,d,e
,Int64,Int64,Int64,Int64,Int64
1,1,1,1,1,1
2,2,2,2,2,2
3,3,3,3,3,3


,a,b,c,d,e
,Int64,Int64,Int64,Int64,Int64
1,100,100,1,1,1
2,2,2,2,2,2
3,3,3,3,3,3
